In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score
import time

In [2]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED) if torch.cuda.is_available() else None
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

In [3]:
DATASET_PATH = "../Dataset"
BATCH_SIZE = 128
IMAGE_SIZE = (224, 224)
NUM_WORKERS = 8  # Adjust based on your CPU cores
PIN_MEMORY = True

In [4]:
transform_for_stats = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_data_stats = ImageFolder(root=os.path.join(DATASET_PATH, "Train"), transform=transform_for_stats)
train_loader_stats = DataLoader(train_data_stats, batch_size=64, shuffle=False, num_workers=4)

mean = torch.zeros(3)
std = torch.zeros(3)
num_batches = 0

for batch, _ in train_loader_stats:
    mean += batch.mean(dim=(0, 2, 3))
    std += batch.std(dim=(0, 2, 3))
    num_batches += 1

mean /= num_batches
std /= num_batches

print(f"Calculated mean: {mean}")
print(f"Calculated std: {std}")

Calculated mean: tensor([0.4671, 0.3818, 0.3416])
Calculated std: tensor([0.2814, 0.2570, 0.2542])


In [4]:
train_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4671, 0.3818, 0.3416], std=[0.2814, 0.2570, 0.2542])
])

test_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4671, 0.3818, 0.3416], std=[0.2814, 0.2570, 0.2542])
])


In [5]:
print("Loading datasets...")
train_data = ImageFolder(root=os.path.join(DATASET_PATH, "Train"), transform=train_transforms)
val_data = ImageFolder(root=os.path.join(DATASET_PATH, "Validation"), transform=test_transforms)
test_data = ImageFolder(root=os.path.join(DATASET_PATH, "Test"), transform=test_transforms)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(val_data)}')
print(f'Number of testing examples: {len(test_data)}')

Loading datasets...
Number of training examples: 140002
Number of validation examples: 39428
Number of testing examples: 10905


In [6]:
train_loader = DataLoader(
    train_data, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    persistent_workers=True if NUM_WORKERS > 0 else False,
    prefetch_factor=4 if NUM_WORKERS > 0 else None,
    drop_last=True
)

val_loader = DataLoader(
    val_data, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    persistent_workers=True if NUM_WORKERS > 0 else False,
    pin_memory=PIN_MEMORY,
    prefetch_factor=4 if NUM_WORKERS > 0 else None
)

test_loader = DataLoader(
    test_data, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    persistent_workers=True if NUM_WORKERS > 0 else False,
    pin_memory=PIN_MEMORY,
    prefetch_factor=4 if NUM_WORKERS > 0 else None
)


In [7]:
class DeepFakeRestNetDetector(nn.Module):
    def __init__(self):
        super().__init__()
        # Load pretrained ResNet50
        self.backbone = torchvision.models.resnet50(weights='DEFAULT')
        in_features = self.backbone.fc.in_features
        
        # Simple classifier head - minimal modification
        self.backbone.fc = nn.Sequential(
            nn.Linear(in_features, 1)
        )

    def forward(self, x):
        return self.backbone(x)

In [8]:
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    all_preds = []
    all_probs = []
    all_targets = []
    
    # For calculating running accuracy during evaluation
    correct = 0
    total = 0
    
    # Create tqdm progress bar
    start_time = time.time()
    pbar = tqdm(loader, desc="Evaluating")
    
    for images, labels in pbar:
        batch_size = images.size(0)
        images = images.to(device, non_blocking=True)
        if hasattr(torch, 'channels_last') and torch.cuda.is_available():
            images = images.to(memory_format=torch.channels_last)
            
        # Forward pass
        outputs = model(images)
        probs = torch.sigmoid(outputs).squeeze()
        
        # Handle case when batch size is 1
        if probs.ndim == 0:
            probs = probs.unsqueeze(0)
            
        # Get binary predictions
        preds = (probs >= 0.5).int()
        
        # Calculate batch accuracy
        batch_correct = (preds.cpu() == labels).sum().item()
        correct += batch_correct
        total += batch_size
        
        # Store for final metrics
        all_probs.append(probs.cpu().numpy())
        all_targets.append(labels.cpu().numpy())
        
        # Update progress bar with current accuracy
        current_accuracy = correct / total
        pbar.set_postfix({
            'Acc': f'{current_accuracy:.4f}',
            'Imgs/sec': f'{total / (time.time() - start_time):.1f}'
        })
    
    # Final evaluation
    all_probs = np.concatenate(all_probs)
    all_targets = np.concatenate(all_targets)
    
    # Calculate metrics
    final_accuracy = accuracy_score(all_targets, (all_probs >= 0.5).astype(int))
    auc = roc_auc_score(all_targets, all_probs)
    
    total_time = time.time() - start_time
    print(f"\nEvaluation completed in {total_time:.2f} seconds")
    print(f"Average time per image: {total_time/len(all_targets):.4f} seconds")
    
    return {
        'accuracy': final_accuracy,
        'auc': auc
    }

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
    
# Create model
model = DeepFakeRestNetDetector().to(device)

Using device: cpu


In [10]:
if hasattr(torch, 'compile') and torch.cuda.is_available():
        try:
            model = torch.compile(model)
            print("Model compiled successfully")
        except Exception as e:
            print(f"Model compilation failed: {e}. Continuing with standard model.")
    
    # Start evaluation
print("-" * 50)
print("Starting evaluation on test set...")
results = evaluate(model, test_loader, device)
    
print("\n===== RESULTS =====")
print(f"Final Accuracy: {results['accuracy']:.4f}")
print(f"AUC-ROC Score: {results['auc']:.4f}")
print("-" * 50)

--------------------------------------------------
Starting evaluation on test set...


Evaluating: 100%|██████████| 86/86 [20:50<00:00, 14.54s/it, Acc=0.5075, Imgs/sec=8.7] 



Evaluation completed in 1250.88 seconds
Average time per image: 0.1147 seconds

===== RESULTS =====
Final Accuracy: 0.5075
AUC-ROC Score: 0.5173
--------------------------------------------------
